In [ ]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from torchvision.models import densenet121
import math

In [ ]:
modellr = 1e-4
BATCH_SIZE = 32
EPOCHS = 10
#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = 'cpu'

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
 
])
transform_validation = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
# training set and validation set see google drive
dataset_train = datasets.ImageFolder(r"C:\Users\lovel\Downloads\rm + nonrm", transform)
dataset_validation = datasets.ImageFolder(r"C:\Users\lovel\Downloads\val_set", transform_validation)
# read data
#print(dataset_train.imgs)
print(len(dataset_train),len(dataset_validation))
print(dataset_train.class_to_idx,dataset_validation.class_to_idx) 
# import data
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset_validation, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
criterion = nn.CrossEntropyLoss()
model_ft = densenet121(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 2)
model_ft.to(DEVICE)

optimizer = optim.Adam(model_ft.parameters(), lr=modellr)
 
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 50))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew

In [ ]:
model = torch.load("model.pth")

In [ ]:
train_prob = []
train_label = []

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sum_loss = 0
    total_num = len(train_loader.dataset)
    print('total number of images:',total_num, 'total number of batches:',len(train_loader))
    for batch_idx, (data, target) in enumerate(train_loader):
        print('batch:{}/{}'.format(str(batch_idx),str(len(train_loader))))
        data, target = Variable(data).to(device), Variable(target).to(device)
        #print(target.numpy())
        train_label.append(target)
        output = model(data)
        loss = criterion(output, target)
        #_, pred = torch.max(output.data, 1)
        train_prob.append(torch.sigmoid(output.data).numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss
    avgloss = sum_loss / len(train_loader)
    return avgloss

In [ ]:
# validation
def val(model, device, validation_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(validation_loader.dataset)
    print('total number of images:',total_num, 'total number of batches:',len(validation_loader))
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            output = model(data)
            loss = criterion(output, target)
            _, pred = torch.max(output.data, 1)
            correct += torch.sum(pred == target)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(validation_loader)
        print('\nVal set: Accuracy: {}/{} ({:.0f}%)\n'.format(correct, len(validation_loader.dataset), 100 * acc))
    return avgloss


In [ ]:
def save_weight(val_loss):
    torch.save(model_ft, 'model.pth')

In [ ]:
min_val_loss = math.inf

for epoch in range(EPOCHS):
    #torch.cuda.empty_cache()
    train_loss =train(model_ft, DEVICE, train_loader, optimizer, epoch)
    val_loss = val(model_ft, DEVICE, validation_loader)
    if val_loss < min_val_loss:
        save_weight(val_loss)
        min_val_loss = val_loss
    print('EPOCH {}/{} \t train loss {} \t val loss {}\n'.format(epoch + 1, EPOCHS,train_loss,val_loss))

In [ ]:
# getting training probability for Youden index
import numpy as np
prob = []
np.set_printoptions(suppress=True)
np.set_printoptions(precision=6)
print(train_prob)
for i in range(len(train_prob)):
    for j in range(len(train_prob[i])):
        prob.append(train_prob[i][j][1])
print(prob)

In [ ]:
# getting training labels for Youden index
len(train_label)
label = []
for i in range(len(train_label)):
    for j in range(len(train_label[i])):
        label.append(train_label[i][j])
#print(label)
#print(len(label))

In [ ]:
# stupid way for getting testing groundtruth
import glob
import os
path1 = r'C:\Users\lovel\Downloads\mange\*'
files1 = glob.glob(path1)
mange = []
path2 = r'C:\Users\lovel\Downloads\non mange\*'
files2 = glob.glob(path2)
nonmange = []
for i in range(len(files1)):
    filename = os.path.basename(files1[i])
    print(filename)
    mange.append(filename)
    #print(mange)

for j in range(len(files2)):
    filename = os.path.basename(files2[j])
    print(filename)
    nonmange.append(filename)   

In [ ]:
import numpy
groundtruth = []
path3 = r'C:\Users\lovel\Downloads\test\*'
files3 = glob.glob(path3)
#print(files3)

for k in range(len(files3)):
    filename = os.path.basename(files3[k])
    #print(filename)
    
    #mange.append(filename)
    if filename in mange:
        groundtruth.append(1)
    elif filename in nonmange:
        groundtruth.append(0)

print(groundtruth)
y_true = numpy.array(groundtruth)

In [ ]:
import torch.utils.data.distributed
import torchvision.transforms as transforms
from PIL import Image
from torch.autograd import Variable
import os
import numpy as np
# testing
classes=('Nonmange','Mange')
transform_test = transforms.Compose([
         transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
prob = []
predict = [] 
output = []
#DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE = 'cpu'
model = torch.load("model.pth")
model.eval()
model.to(DEVICE)

path=r'C:/Users/lovel/Downloads/test/'
testList=os.listdir(path)
i = 0
for file in testList:
        print(file)
        img=Image.open(path+file)
        img=transform_test(img)
        img.unsqueeze_(0)
        img = Variable(img).to(DEVICE)
        out=model(img)
        # Predict
        x, pred = torch.max(torch.sigmoid(out.data), 1)
        output.append(torch.sigmoid(out.data).numpy())
        #print(x)
        groundtruth.append(classes[pred.data.item()])
        print('Image Name:{},predict:{}'.format(file,classes[pred.data.item()]))
        
        if pred.numpy() != 1:
            x = 1-x.numpy()
        print(x)
        prob.append(x)
        pred = pred.numpy()
        predict.append(pred)
        i += 1
print(prob)

y_score = torch.tensor((prob))
print(y_score)
#row_sums = torch.sum(y_score, 1)
#print(row_sums)
#row_sums = row_sums.repeat(1, 2)
#y_score = torch.div( y_score , row_sums )

y_pred = np.array(predict)
print(y_pred)
print(output)

In [ ]:
# getting testing prob for Youden index
mange = []
#nonmange = []
#print(output)
for i in range(len(output)):
    #print(output[i][0][0])
    #nonmange.append(output[i][0][0])
    mange.append(output[i][0][1])
print(mange)
print(len(mange))
#print(nonmange)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(
    y_true, y_score.flatten(),
#     name=f"{class_of_interest} vs the rest",
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred.flatten()).ravel()
(tn, fp, fn, tp)

In [ ]:
from sklearn.metrics import f1_score
tn = 209
fp = 4
fn = 1
tp = 4
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score
print("Precision:", precision_score(y_true, y_pred.flatten()))
#precision = tp / (tp + fp)
print("Recall:", recall_score(y_true, y_pred.flatten()))
#recall = tp / (tp + fn)
print("F1 score:", f1_score(y_true, y_pred))
#print("F1 score:", 2 * (precision * recall) / (precision + recall))
print("F2 score:", fbeta_score(y_true, y_pred, beta=2))
print("accuracy:", (tp+tn)/(tp+tn+fp+fn))
#print("Binary expected cost:",(0. * tp + 1. * fp + 5. * fn + 0. * tn) / (tp + tn + fp + fn))